In [2]:
# ============================================================
# CAFA 6: Protein Function Prediction - 3 models for each aspect
# ============================================================
# Idea:
# 1. Train 3 separate models for GO aspects
# 2. Propagate all ancestors of GO terms
# 3. Use different threshold for each aspect
# ============================================================

import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from pathlib import Path
from collections import defaultdict
import random

# ------------------------------------------------------------
# 1. Configuration
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

training_config = {
    "batch_size": 128,
    "epochs": 30,
    "hidden_layer_one": 1024,
    "hidden_layer_two": 512,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "dropout_rate": 0.3,
    "train_val_split": 0.9,
    "thresholds": {
        'C': 0.02,   # CCO
        'F': 0.05,   # MFO
        'P': 0.015   # BPO
    },
    "min_predictions_per_protein": 25,
}

aspect_names = {
    'C': 'Cellular Component (CCO)',
    'F': 'Molecular Function (MFO)',
    'P': 'Biological Process (BPO)'
}

print("Current config:", training_config)


# --------------------------------------------------------------------------------------
# 2. TỰ ĐỘNG TÌM ĐƯỜNG DẪN DỮ LIỆU
# --------------------------------------------------------------------------------------
print("\nSearching for data paths...")

embedding_directory = None
for root, dirs, files in os.walk('/kaggle/input'):
    if 'protein_embeddings.npy' in files:
        embedding_directory = root
        break

if not embedding_directory:
    raise FileNotFoundError("Embedding file not found! Please add 'CAFA6 Protein Embeddings' dataset.")

embedding_file_path = os.path.join(embedding_directory, 'protein_embeddings.npy')
protein_id_file_path = os.path.join(embedding_directory, 'protein_ids.csv')
print("Embedding file located at:", embedding_file_path)

competition_base_path = Path('/kaggle/input/cafa-6-protein-function-prediction')
if not (competition_base_path / 'Train').exists():
    for root, dirs, files in os.walk('/kaggle/input'):
        if 'train_terms.tsv' in files:
            competition_base_path = Path(root).parent
            if 'Train' not in str(competition_base_path):
                competition_base_path = Path(root)
            break

train_data_path = competition_base_path / 'Train' if (competition_base_path / 'Train').exists() else competition_base_path
test_data_path = competition_base_path / 'Test' if (competition_base_path / 'Test').exists() else competition_base_path
print("Training data found at:", train_data_path)

# --------------------------------------------------------------------------------------
# 3. LOAD DỮ LIỆU & parse GO hierarchy
# --------------------------------------------------------------------------------------
print("\nLoading training data...")

# read training terms
training_terms = pd.read_csv(train_data_path / 'train_terms.tsv', sep='\t')
print("Total annotations:", len(training_terms))
print("Annotations per aspect:\n", training_terms['aspect'].value_counts())

# load embeddings
embeddings_matrix = np.load(embedding_file_path)
protein_id_dataframe = pd.read_csv(protein_id_file_path)

if 'protein_id' in protein_id_dataframe.columns:
    protein_ids = protein_id_dataframe['protein_id'].astype(str).values
else:
    protein_ids = protein_id_dataframe.iloc[:, 0].astype(str).values

# map protein id to embedding index
protein_id_to_index = {pid: i for i, pid in enumerate(protein_ids)}
embeddings_lookup = {pid: embeddings_matrix[i] for pid, i in protein_id_to_index.items()}
input_dimension = embeddings_matrix.shape[1]
print("Loaded", len(protein_ids), "embeddings with dimension:", input_dimension)

print("\nParsing GO hierarchy...")
go_parent_terms = defaultdict(list)
go_child_terms = defaultdict(list)

with open(train_data_path / 'go-basic.obo', 'r') as f:
    current_term = None
    is_obsolete = False
    for line in f:
        line = line.strip()
        if line == "[Term]":
            current_term = None
            is_obsolete = False
        elif line.startswith("id: GO:"):
            current_term = line.split("id: ")[1]
        elif line.startswith("is_obsolete: true"):
            is_obsolete = True
        elif current_term and not is_obsolete:
            if line.startswith("is_a:"):
                parent_term = line.split("is_a: ")[1].split(" !")[0].strip()
                go_parent_terms[current_term].append(parent_term)
                go_child_terms[parent_term].append(current_term)
            elif line.startswith("relationship: part_of"):
                parent_term = line.split("part_of ")[1].split(" !")[0].strip()
                go_parent_terms[current_term].append(parent_term)
                go_child_terms[parent_term].append(current_term)

print("Number of GO terms with parent:", len(go_parent_terms))


# --------------------------------------------------------------------------------------
# 4. MODEL DEFINITION
# --------------------------------------------------------------------------------------
print("\nDefining model...")

class ProteinFunctionMLP(nn.Module):
    # simple MLP for protein function prediction
    def __init__(self, input_dimension, output_classes, hidden_layer_one=1024, hidden_layer_two=512, dropout_rate=0.3):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dimension, hidden_layer_one),
            nn.BatchNorm1d(hidden_layer_one),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_layer_one, hidden_layer_two),
            nn.BatchNorm1d(hidden_layer_two),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_layer_two, output_classes)
        )

    def forward(self, x):
        return self.layers(x)

def get_all_parent_terms(go_term, cache={}):
    # return all ancestor terms of a GO term
    if go_term in cache:
        return cache[go_term]
    ancestor_terms = set()
    stack = [go_term]
    while stack:
        current = stack.pop()
        for parent in go_parent_terms.get(current, []):
            if parent not in ancestor_terms:
                ancestor_terms.add(parent)
                stack.append(parent)
    cache[go_term] = ancestor_terms
    return ancestor_terms

print("Model architecture ready")

# ============================================================
# 5. TRAIN 3 ASPECT-SPECIFIC MODELS
# ============================================================
print("\nTraining 3 separate models for each aspect...")

def set_random_seed(seed=42):
    # fix seed for reproducibility
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# collect protein ids
training_protein_ids = training_terms['EntryID'].unique()
valid_training_proteins = [pid for pid in training_protein_ids if pid in protein_id_to_index]

aspect_models = {}
aspect_label_encoders = {}

for aspect in ['C', 'F', 'P']:
    set_random_seed(42)
    
    aspect_name = aspect_names[aspect]
    print("\n" + "="*60)
    print("Training model for:", aspect_name)
    
    aspect_dataframe = training_terms[training_terms['aspect'] == aspect]
    protein_to_go_terms = aspect_dataframe.groupby('EntryID')['term'].apply(list).to_dict()
    proteins_for_aspect = [pid for pid in valid_training_proteins if pid in protein_to_go_terms]
    
    labels_per_protein = [protein_to_go_terms[pid] for pid in proteins_for_aspect]
    label_binarizer = MultiLabelBinarizer(sparse_output=False)
    y_labels = label_binarizer.fit_transform(labels_per_protein)
    num_classes = len(label_binarizer.classes_)
    
    X_data = np.array([embeddings_lookup[pid] for pid in proteins_for_aspect], dtype=np.float32)
    y_data = y_labels.astype(np.float32)
    
    dataset = TensorDataset(torch.from_numpy(X_data), torch.from_numpy(y_data))
    
    indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(indices, train_size=training_config['train_val_split'], random_state=42, shuffle=True)
    
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    
    train_loader = DataLoader(train_dataset, batch_size=training_config['batch_size'], shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=training_config['batch_size']*2, shuffle=False, num_workers=0)
    
    model = ProteinFunctionMLP(input_dimension, num_classes, training_config['hidden_layer_one'], training_config['hidden_layer_two'], training_config['dropout_rate']).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=training_config['learning_rate'], weight_decay=training_config['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    loss_function = nn.BCEWithLogitsLoss()
    
    best_validation_loss = float('inf')
    best_model_state = None

    for epoch in range(training_config['epochs']):
        # training phase
        model.train()
        total_train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * batch_x.size(0)
        
        average_train_loss = total_train_loss / len(train_loader.dataset)
        
        # validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                total_val_loss += loss_function(outputs, batch_y).item() * batch_x.size(0)
        
        average_val_loss = total_val_loss / len(val_loader.dataset)
        scheduler.step(average_val_loss)
        
        # save best model
        if average_val_loss < best_validation_loss:
            best_validation_loss = average_val_loss
            best_model_state = model.state_dict().copy()
        
        # print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print("Epoch:", epoch+1, "/", training_config['epochs'], 
                  "| Train Loss:", round(average_train_loss, 6), 
                  "| Val Loss:", round(average_val_loss, 6))

    model.load_state_dict(best_model_state)
    aspect_models[aspect] = model
    aspect_label_encoders[aspect] = label_binarizer
    
    # cleanup
    del train_loader, val_loader, train_dataset, val_dataset, dataset, X_data, y_data
    torch.cuda.empty_cache()
    gc.collect()

print("\nAll 3 aspect-specific models trained successfully!")


# --------------------------------------------------------------------------------------
# 6. LOAD TEST DATA
# --------------------------------------------------------------------------------------
print("\nLoading test data...")

test_protein_ids = []
fasta_file_path = test_data_path / 'testsuperset.fasta'

# check if fasta file exists, otherwise search in input directory
if not fasta_file_path.exists():
    for root, dirs, files in os.walk('/kaggle/input'):
        if 'testsuperset.fasta' in files:
            fasta_file_path = Path(root) / 'testsuperset.fasta'
            break

# read protein ids from fasta file
with open(fasta_file_path, 'r') as fasta_file:
    for line in fasta_file:
        if line.startswith('>'):
            parts = line[1:].split('|')
            protein_id = parts[1] if len(parts) >= 2 else line[1:].strip().split()[0]
            test_protein_ids.append(protein_id)

# remove duplicates
test_protein_ids = list(set(test_protein_ids))

# keep only proteins that have embeddings
valid_test_protein_ids = [pid for pid in test_protein_ids if pid in protein_id_to_index]

print("Number of test proteins with embeddings:", len(valid_test_protein_ids))


# --------------------------------------------------------------------------------------
# 7. ENSEMBLE PREDICTION
# --------------------------------------------------------------------------------------
print("\nRunning ensemble prediction...")
all_predictions = []

for aspect in ['C', 'F', 'P']:
    aspect_name = aspect_names[aspect]
    print("\nPredicting for aspect:", aspect_name)
    
    model = aspect_models[aspect]
    label_binarizer = aspect_label_encoders[aspect]
    threshold_value = training_config['thresholds'][aspect]
    model.eval()
    
    test_embeddings_array = np.array([embeddings_lookup[pid] for pid in valid_test_protein_ids], dtype=np.float32)
    test_loader = DataLoader(TensorDataset(torch.from_numpy(test_embeddings_array)), batch_size=2048, shuffle=False)
    
    batch_index = 0
    with torch.no_grad():
        for (batch_x,) in tqdm(test_loader, desc=f"Predicting ({aspect})"):
            outputs = model(batch_x.to(device))
            probabilities = torch.sigmoid(outputs).cpu().numpy()
            current_batch_size = probabilities.shape[0]
            
            for i in range(current_batch_size):
                protein_id = valid_test_protein_ids[batch_index + i]
                scores = probabilities[i]
                
                selected_indices = np.where(scores >= threshold_value)[0]
                if len(selected_indices) < training_config['min_predictions_per_protein']:
                    selected_indices = np.argsort(scores)[-training_config['min_predictions_per_protein']:]
                
                for idx in selected_indices:
                    all_predictions.append({
                        'protein_id': protein_id,
                        'go_term': label_binarizer.classes_[idx],
                        'score': float(scores[idx])
                    })
            
            batch_index += current_batch_size
    
    del test_embeddings_array, test_loader
    torch.cuda.empty_cache()

predictions_dataframe = pd.DataFrame(all_predictions)
print("Total raw predictions:", len(predictions_dataframe))


# --------------------------------------------------------------------------------------
# 8. GO TERM PROPAGATION
# --------------------------------------------------------------------------------------
print("\nPropagating GO terms...")

ancestor_cache = {}

def get_cached_ancestors(go_term):
    # return all ancestor terms of a GO term
    if go_term in ancestor_cache:
        return ancestor_cache[go_term]
    ancestors = set()
    stack = [go_term]
    while stack:
        current = stack.pop()
        for parent in go_parent_terms.get(current, []):
            if parent not in ancestors:
                ancestors.add(parent)
                stack.append(parent)
    ancestor_cache[go_term] = ancestors
    return ancestors

# cache ancestors for all predicted terms
for term in tqdm(predictions_dataframe['go_term'].unique(), desc="Caching ancestors"):
    get_cached_ancestors(term)
print("Cached ancestor terms:", len(ancestor_cache))

submission_file_name = 'submission.tsv'
print("Writing results to:", submission_file_name)

total_written_rows = 0
with open(submission_file_name, 'w') as output_file:
    for protein_id, group in tqdm(predictions_dataframe.groupby('protein_id'), desc="Propagate and write"):
        term_scores = {}
        terms = group['go_term'].values
        scores = group['score'].values
        
        for term, score in zip(terms, scores):
            term_scores[term] = max(term_scores.get(term, 0), score)
            for ancestor in ancestor_cache.get(term, set()):
                term_scores[ancestor] = max(term_scores.get(ancestor, 0), score)
        
        for term, score in sorted(term_scores.items(), key=lambda x: -x[1]):
            output_file.write(f"{protein_id}\t{term}\t{score:.4f}\n")
            total_written_rows += 1
        
        del term_scores

del predictions_dataframe
gc.collect()

print("\nPropagation finished. Total rows written:", total_written_rows)



Using device: cuda
Current config: {'batch_size': 128, 'epochs': 30, 'hidden_layer_one': 1024, 'hidden_layer_two': 512, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'dropout_rate': 0.3, 'train_val_split': 0.9, 'thresholds': {'C': 0.02, 'F': 0.05, 'P': 0.015}, 'min_predictions_per_protein': 25}

Searching for data paths...
Embedding file located at: /kaggle/input/cafa6-protein-embeddings-esm2/protein_embeddings.npy
Training data found at: /kaggle/input/cafa-6-protein-function-prediction/Train

Loading training data...
Total annotations: 537027
Annotations per aspect:
 aspect
P    250805
C    157770
F    128452
Name: count, dtype: int64
Loaded 287001 embeddings with dimension: 1280

Parsing GO hierarchy...
Number of GO terms with parent: 40119

Defining model...
Model architecture ready

Training 3 separate models for each aspect...

Training model for: Cellular Component (CCO)
Epoch: 1 / 30 | Train Loss: 0.017597 | Val Loss: 0.004727
Epoch: 5 / 30 | Train Loss: 0.003795 | Val Loss: 0

Predicting (C):   0%|          | 0/110 [00:00<?, ?it/s]


Predicting for aspect: Molecular Function (MFO)


Predicting (F):   0%|          | 0/110 [00:00<?, ?it/s]


Predicting for aspect: Biological Process (BPO)


...

Total raw predictions: 20326264

Propagating GO terms...


Caching ancestors:   0%|          | 0/25550 [00:00<?, ?it/s]

Cached ancestor terms: 25550
Writing results to: submission.tsv


Propagate and write:   0%|          | 0/224309 [00:00<?, ?it/s]


Propagation finished. Total rows written: 71665570


In [3]:
from IPython.display import FileLink
FileLink('submission.tsv')

/kaggle/working/submission.tsv